# Single Estimation

This notebook shows the full pipeline for estimating 6D pose of an object given 
RGB image $\mathcal{I}$ and matched depth map $\mathcal{D}$ and Object segmentation mask $\mathcal{M}$.

This is shown the following steps

In [1]:
# Imports
from matplotlib import pyplot as plt
import os
from pathlib import Path
# import numpy as np
import torch
# import torch.utils.model_zoo
import math

# LatentFusion
from latentfusion.recon.inference import Observation
from latentfusion.datasets.realsense import RealsenseDataset
import latentfusion.visualization as viz
from latentfusion.augment import gan_denormalize
from latentfusion import meshutils
from latentfusion import augment
from latentfusion.recon.inference import LatentFusionModel
from latentfusion.three.orientation import evenly_distributed_quats
import latentfusion.pose.estimation as pe

2024-10-03 18:50.02 [warning  ] could not import PCL           [latentfusion.pointcloud]


In [2]:
# Setup global environment
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
MOPED_PATH = Path('datasets/moped')
num_ref_views = 8 # How many reference images to use?

Using device: cuda:0


In [3]:
# Load pre-trained model
checkpoint = torch.load('weights/latentfusion-release.pth')
model = LatentFusionModel.from_checkpoint(checkpoint, device)

/tmp/ipykernel_62246/3030555503.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('weights/latentfusion-release.pth')
2024-10-03 18:50.10 [info    

## Step 1: Create latent representation of the new object.

In [4]:
object_id = 'toy_plane' # 
test_frame = 20

# Define path variables
# TODO check if we can get away without using the ground truth
input_scene_dir = MOPED_PATH / object_id / 'reference' # Ground-truth
target_scene_dir = MOPED_PATH / object_id / 'evaluation' # Input

# Why are we using object's point cloud?
pointcloud_path = input_scene_dir / 'integrated_registered_processed.obj'
obj = meshutils.Object3D(pointcloud_path)
pointcloud = torch.tensor(obj.vertices, dtype=torch.float32) # May not be used anywhere?
diameter = obj.bounding_diameter
object_scale = 1.0 / diameter
object_scale_to_meters = 1.0 / object_scale

print(f"diameter: {diameter}")
print(f"object_scale: {object_scale}")
print(f"object_scale_to_meters: {object_scale_to_meters}")


diameter: 0.3425245885532891
object_scale: 2.919498434327504
object_scale_to_meters: 0.3425245885532891


To check how this pointcloud looks like, use Cloud compare
```/usr/bin/CloudCompare```

In [ ]:
# TODO find out why we need both .obj file and how it was made?
# TODO the function of RealsenseDataset class??
# Make listof paths
input_paths = [x for x in input_scene_dir.iterdir() if x.is_dir()]
#print(input_paths[:2]) # [PosixPath('datasets/moped/toy_plane/reference/06')]
input_dataset = RealsenseDataset(input_paths,
                                 image_scale=1.0,
                                 object_scale=object_scale,
                                 odometry_type='open3d')

In [ ]:
target_paths = sorted([x for x in target_scene_dir.iterdir() if x.is_dir()])
target_dataset = RealsenseDataset(target_paths,
                                  image_scale=1.0,
                                  object_scale=object_scale, # Can we just use 1.0 here??
                                  odometry_type='open3d',
                                  use_registration=True)
                                  

## Step 1: Create latent representation of the new object.

## Step 2: For given reference, find coarse estimation

## Step 3: For the coarse estimates, obtain fine estimation